In [7]:
import os
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms
from piq import ssim


def psnr(input, target, data_range=1.0):
    mse = torch.mean((input - target) ** 2)
    if mse == 0:
        return float('inf')
    return 10 * torch.log10((data_range ** 2) / mse)

image_folder = 'C:/Users/piai/Desktop/Domestic/retinex_GAN/input/test'
target_folder = 'C:/Users/piai/Desktop/Domestic/retinex_GAN/output_result/test' 

image_files = [os.path.join(image_folder, file) 
               for file in os.listdir(image_folder) 
               if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

psnr_list = []
ssim_list = []

preprocess = transforms.Compose([transforms.ToTensor()])


for image_file in image_files:
    image = Image.open(image_file).convert("RGB")
    image = preprocess(image)
    target_file = os.path.join(target_folder, os.path.basename(image_file))
    if os.path.exists(target_file):
        target_image = Image.open(target_file).convert("RGB")
        target_image = preprocess(target_image)
        image = image.unsqueeze(0)  # 차원 추가
        target_image = target_image.unsqueeze(0)  # 차원 추가
        psnr_value = psnr(image, target_image, data_range=1.0)
        psnr_list.append(psnr_value)
        ssim_value = ssim(image, target_image, data_range=1.0)
        ssim_list.append(ssim_value)
        print("{} - PSNR : {} SSIM: {}".format(image_file.split('/')[-1],psnr_value,ssim_value))
        
psnr_mean = torch.mean(torch.stack(psnr_list))
ssim_mean = torch.mean(torch.stack(ssim_list))
print("Average - PSNR : {} SSIM: {}".format(psnr_mean.item(),ssim_mean.item()))


test\2_img_.png - PSNR : 24.50922966003418 SSIM: 0.9293501973152161
test\im_f8_.jpg - PSNR : 16.308677673339844 SSIM: 0.9311525225639343
test\im_f9_.jpg - PSNR : 18.857975006103516 SSIM: 0.8811513781547546
Average - PSNR : 19.8919620513916 SSIM: 0.913884699344635
